In [50]:
import pandas as pd

In [51]:
prices = pd.read_csv('BTC-2018min.csv')
tweets = pd.read_csv('BTC_Tweets_Updated.csv')

In [52]:
prices.head()

,unix,date,symbol,open,high,low,close,Volume BTC,Volume USD
0,1546300740,2018-12-31 23:59:00,BTC/USD,3689.26,3693.30,3689.26,3693.30,9.838855,36337.841473
1,1546300680,2018-12-31 23:58:00,BTC/USD,3689.26,3691.35,3689.26,3691.35,0.560000,2067.156000
2,1546300620,2018-12-31 23:57:00,BTC/USD,3688.83,3689.26,3688.83,3689.26,0.560833,2069.059602
3,1546300560,2018-12-31 23:56:00,BTC/USD,3687.87,3689.65,3686.92,3686.92,7.610240,28058.346208
4,1546300500,2018-12-31 23:55:00,BTC/USD,3688.28,3688.85,3685.00,3688.85,7.665703,28277.629729


In [53]:
tweets.head()

,id,Date,Tweet,Screen_name,Source,Link,Sentiment,sent_score,New_Sentiment_Score,New_Sentiment_State,BERT Labels
0,0,Fri Mar 23 00:40:32 +0000 2018,"RT @ALXTOKEN: Paul Krugman, Nobel Luddite. I h...",myresumerocket,[],"<a href=""http://twitter.com"" rel=""nofollow"">Tw...",['neutral'],0.0,0.000000,0.0,1
1,1,Fri Mar 23 00:40:34 +0000 2018,@lopp @_Kevin_Pham @psycho_sage @naval But @Pr...,BitMocro,[u'Bitcoin'],"<a href=""http://twitter.com/download/android"" ...",['neutral'],0.0,0.000000,0.0,0
2,2,Fri Mar 23 00:40:35 +0000 2018,RT @tippereconomy: Another use case for #block...,hojachotopur,"[u'blockchain', u'Tipper', u'TipperEconomy']","<a href=""http://twitter.com"" rel=""nofollow"">Tw...",['positive'],1.0,0.136364,1.0,0
3,3,Fri Mar 23 00:40:36 +0000 2018,free coins https://t.co/DiuoePJdap,denies_distro,[],"<a href=""http://twitter.com"" rel=""nofollow"">Tw...",['positive'],1.0,0.400000,1.0,0
4,4,Fri Mar 23 00:40:36 +0000 2018,RT @payvxofficial: WE are happy to announce th...,aditzgraha,[],"<a href=""http://twitter.com/download/android"" ...",['positive'],1.0,0.468182,1.0,0


In [54]:
tweets['datetime'] = pd.to_datetime(tweets['Date'])
tweets['datetime'] = tweets['datetime'].dt.tz_convert(None)
tweets['minute'] = tweets['datetime'].dt.floor('T')
volume = tweets.groupby('minute').size().rename('msg_volume')
sentiment = tweets.groupby('minute')['sent_score'].mean().rename('avg_sentiment')
tweets_minute = pd.merge(volume, sentiment, on='minute')
tweets_minute

/var/folders/cl/7jg_ds7n04dc07vf274tlhk40000gn/T/ipykernel_51450/462288409.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  tweets['datetime'] = pd.to_datetime(tweets['Date'])


,msg_volume,avg_sentiment
minute,,
2018-03-23 00:40:00,33,0.575758
2018-03-23 00:41:00,95,0.284211
2018-03-23 00:42:00,114,0.368421
2018-03-23 00:43:00,106,0.358491
2018-03-23 00:44:00,93,0.387097
...,...,...
2018-03-23 08:51:00,110,0.372727
2018-03-23 08:52:00,111,0.351351
2018-03-23 08:53:00,138,0.376812


In [55]:
prices['date'] = prices['date'].astype(str)
prices['date'] = pd.to_datetime(prices['date'])
prices['minute'] = prices['date'].dt.floor('T')
prices_minute = prices.groupby('minute').agg({'close':'last'}).rename(columns={'close':'close_price'})
prices_minute['price_change'] = prices_minute['close_price'].diff()
prices_minute

,close_price,price_change
minute,,
2018-01-01 00:01:00,13890.43,NaN
2018-01-01 00:02:00,13847.51,-42.92
2018-01-01 00:03:00,13823.19,-24.32
2018-01-01 00:04:00,13803.20,-19.99
2018-01-01 00:05:00,13737.93,-65.27
...,...,...
2018-12-31 23:55:00,3688.85,4.00
2018-12-31 23:56:00,3686.92,-1.93
2018-12-31 23:57:00,3689.26,2.34


In [56]:
preprocessed_data = pd.merge(tweets_minute, prices_minute, on='minute', how='inner')
preprocessed_data

,msg_volume,avg_sentiment,close_price,price_change
minute,,,,
2018-03-23 00:40:00,33,0.575758,8719.43,11.13
2018-03-23 00:41:00,95,0.284211,8701.82,-17.61
2018-03-23 00:42:00,114,0.368421,8703.53,1.71
2018-03-23 00:43:00,106,0.358491,8703.51,-0.02
2018-03-23 00:44:00,93,0.387097,8703.51,0.00
...,...,...,...,...
2018-03-23 08:51:00,110,0.372727,8490.00,-12.00
2018-03-23 08:52:00,111,0.351351,8484.17,-5.83
2018-03-23 08:53:00,138,0.376812,8485.77,1.60
